In [60]:
import sqlite3
import pandas as pd
import numpy as np

In [2]:
#OUT_PATH = 'D:/fvs_calibration/data/interim'

In [123]:
#read in FIA source tables 
DB_PATH = 'D:FIA/FIADB_CA.db'

con = sqlite3.connect(DB_PATH)
# grab fvs_standinit_cond, cond, tree tables
stands = pd.read_sql_query("select STAND_CN, INV_YEAR AS INVYR, REGION, FOREST from FVS_STANDINIT_COND where VARIANT = 'NC'", con)
cond = pd.read_sql_query("select CN AS STAND_CN, PLT_CN, CONDID, INVYR from COND", con)
trees = pd.read_sql_query("select CN AS TRE_CN, PLT_CN, CONDID, INVYR, TREE, STATUSCD, TPA_UNADJ, SPCD, DIA, HT, VOLCFGRS, AGENTCD, STANDING_DEAD_CD from TREE", con)


In [124]:
# use cond key matrix to get plt_cn to match with trees and stand table
cond['INVYR'] = cond['INVYR'].astype('int64')
trees_id = trees.merge(cond, how='left', on=('PLT_CN', 'CONDID', 'INVYR'))
# join with stands with tree records to filter to NC variant
stands['INVYR'] = stands['INVYR'].astype('int64')
stands_cn = stands[['STAND_CN', 'INVYR', 'REGION', 'FOREST']]
trees_filtered = trees_id.merge(stands, how='inner', on=('STAND_CN', 'INVYR'))
trees_filtered.head(1)

,TRE_CN,PLT_CN,CONDID,INVYR,TREE,STATUSCD,TPA_UNADJ,SPCD,DIA,HT,VOLCFGRS,AGENTCD,STANDING_DEAD_CD,STAND_CN,REGION,FOREST
0,23652246010900,23652215010900,1,2001,100,1,6.018046,631.0,7.0,41.0,2.864269,NaN,NaN,23652219010900,5.0,10.0


In [125]:
# let's assign StatusCD manually, so it matches FVS input
trees_filtered.loc[(trees_filtered['STATUSCD'] == 1), 'STATUSCD'] = 1
trees_filtered.loc[(trees_filtered['STATUSCD'] == 2) & (trees_filtered['AGENTCD'].notnull()), 'STATUSCD'] = 6
trees_filtered.loc[(trees_filtered['STATUSCD'] == 3), 'STATUSCD'] = 8
trees_filtered.loc[(trees_filtered['STATUSCD'] == 2) & (trees_filtered['AGENTCD'].isnull()), 'STATUSCD'] = 8
trees_filtered = trees_filtered[['STAND_CN', 'PLT_CN', 'CONDID', 'TRE_CN', 'TREE', 'INVYR', 'STATUSCD', 'TPA_UNADJ', 'REGION', 'FOREST', 'SPCD', 'DIA', 'HT', 'VOLCFGRS']]

In [129]:
trees_filtered['EQUATION'] = '=VolumeEquationNumber()'
trees_filtered['VOLUME_CVTS'] = '=calcFiaVol()'
trees_filtered['VOLUME_CV4'] = '=calcFiaVol()'

In [130]:
# export to excel for use in National Volume Estimator Library (NVEL)
trees_filtered.to_excel('../data/NC_trees_for_NVEL.xlsx')

Calculate volume estimates in NVEL. Save updated workbook as NC_trees_for_NVEL_calc
* ['EQUATION'] = '=VolumeEquationNumber(L2,M2,1,N2,2)' 
* ['VOLUME_CVTS'] = '=calcFiaVol(R2,N2,O2,P2,,"CVTS")'
* ['VOLCALC_CV4'] = '=calcFiaVol(R2,N2,O2,P2,,"CV4")'

In [131]:
# read in nvel volume tree records
volume_calc = pd.read_excel('../data/NC_trees_for_NVEL_calc.xlsx' )
volume_calc.head(5)

,Unnamed: 0,STAND_CN,PLT_CN,CONDID,TRE_CN,TREE,INVYR,STATUSCD,TPA_UNADJ,REGION,FOREST,SPCD,DIA,HT,VOLCFGRS,EQUATION,VOLUME_CVTS,VOLUME_CV4
0,0,23652219010900,23652215010900,1,23652246010900,100,2001,1,6.018046,5,10,631,7.0,41.0,2.864269,500DVEW631,6.742738,2.900000
1,1,23652219010900,23652215010900,1,23652248010900,101,2001,8,0.999188,5,10,202,28.5,131.0,192.310280,500WO2W202,193.607315,176.099976
2,2,23652219010900,23652215010900,1,23652250010900,102,2001,1,6.018046,5,10,631,5.9,44.0,2.132916,500DVEW631,5.143484,2.100000
3,3,23652219010900,23652215010900,1,23652316010900,103,2001,1,6.018046,5,10,631,6.5,42.0,2.501973,500DVEW631,5.995523,2.500000
4,4,23652219010900,23652215010900,1,23652318010900,104,2001,1,6.018046,5,10,631,7.5,45.0,3.706176,500DVEW631,8.293448,3.700000
